In [356]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import matplotlib.pyplot as plt
import hvplot.pandas
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim
from os.path import exists
from urllib.request import urlretrieve
import holoviews as hv
from fiona.crs import from_epsg
from geopy.geocoders import Nominatim
from haversine import haversine, Unit
import leafmap

import warnings
warnings.filterwarnings('ignore')
plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=300, frame_height=500))
#hvplot_defaults = {'tiles':None, 'cmap':'Viridis', 'colorbar':True}


In [357]:
pd.set_option('display.max_colwidth', None) # To avoid truncated display of DataFrames 


In [358]:
df = pd.read_excel("positiondump utc .xlsx", index_col = [0])


In [248]:
#df = df[df["IgnitionOn"] == 1]

In [359]:
df["long"] = df["GPSpoint"].apply(lambda x: x.split(",")[0])
df["lat"] = df["GPSpoint"].apply(lambda x: x.split(",")[-1])

In [361]:
#Convert Longs and Lats to floats. 

def cnvrtCoord(val: str) -> float:
    sgn = +1
    if val[0] == 'S' or val[0] == 'W':
        sgn = -1
    return float(val[1:]) * sgn  

df['long'] = [cnvrtCoord(x) for x in df['long'].tolist()]
df['lat'] = [cnvrtCoord(x) for x in df['lat'].tolist()]  

df.drop("GPSpoint", axis=1, inplace=True)

In [362]:
df['DateTimeOfPosition'] = pd.to_datetime(df['DateTimeOfPosition'])


In [363]:
df

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
647751005,2023-06-14 18:15:03,0,52.203256,5.972714
647742914,2023-06-14 17:17:28,0,52.203289,5.972705
647742890,2023-06-14 17:17:23,0,52.203289,5.972705
647742726,2023-06-14 17:16:23,1,52.203792,5.975125
647742547,2023-06-14 17:15:23,1,52.200175,5.979859
...,...,...,...,...
631418653,2023-04-01 09:22:53,1,53.391285,5.282144
631418504,2023-04-01 09:21:53,1,53.389952,5.274539
631418385,2023-04-01 09:20:53,1,53.389564,5.270365


In [9]:
traj = mpd.Trajectory(df[["DateTimeOfPosition", "long", "lat"]], "Allsetra", x='lat', y='long', t='DateTimeOfPosition', crs=4326)


In [10]:
splitted_traj = mpd.ObservationGapSplitter(traj).split(gap=timedelta(minutes=3))

In [11]:
dfs = [splitted.df for splitted in splitted_traj.trajectories]

combined_dfs = pd.concat(dfs)

In [12]:
combined_dfs

,geometry
DateTimeOfPosition,
2023-04-01 09:19:54,POINT (5.26999 53.38927)
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
2023-04-01 09:23:53,POINT (5.28571 53.39155)
...,...
2023-06-14 17:14:23,POINT (5.97857 52.19321)
2023-06-14 17:15:23,POINT (5.97986 52.20018)
2023-06-14 17:16:23,POINT (5.97513 52.20379)


In [13]:
data = {}
start_addrs = []
end_addrs = []
start_timestamps = []
end_timestamps = []
start_longs_and_lats = []
end_longs_and_lats = []
distances = []
durations = []

for idx, trajectory in enumerate(splitted_traj.trajectories):
        
    start_time = trajectory.df.index[0]
    end_time = trajectory.df.index[-1]
    
    start_timestamps.append(start_time)
    end_timestamps.append(end_time)
    
    #durations.append(str(end_time - start_time).split("days")[-1].split(".")[0])
    durations.append(end_time - start_time)

    start_xy = (trajectory.df["geometry"].x[0], trajectory.df["geometry"].y[0])
    end_xy = (trajectory.df["geometry"].x[-1], trajectory.df["geometry"].y[-1])
    
    
#     start_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[0]))
#     end_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[-1]))

    start_longs_and_lats.append(start_xy)
    end_longs_and_lats.append(end_xy)
    
    distances.append(haversine(start_xy, end_xy, unit=Unit.KILOMETERS))
    
# data["start_trip_addr"] = start_addrs
# data["end_trip_addr"] = end_addrs

data["start_trip_time"] = start_timestamps
data["end_trip_time"] = end_timestamps

data["start_gps_point"] = start_longs_and_lats
data["end_gps_point"] = end_longs_and_lats

data["trip_distance_in_km"] = distances

data["trip_duration"] = durations



In [14]:
splitted_df = pd.DataFrame(data)

In [15]:
splitted_df["trip_duration"].describe()

count                          325
mean     0 days 00:30:45.990769230
std      0 days 00:26:01.865196452
min                0 days 00:00:01
25%                0 days 00:07:29
50%                0 days 00:23:02
75%                0 days 00:47:19
max                0 days 01:55:20
Name: trip_duration, dtype: object

In [16]:
combined_dfs

,geometry
DateTimeOfPosition,
2023-04-01 09:19:54,POINT (5.26999 53.38927)
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
2023-04-01 09:23:53,POINT (5.28571 53.39155)
...,...
2023-06-14 17:14:23,POINT (5.97857 52.19321)
2023-06-14 17:15:23,POINT (5.97986 52.20018)
2023-06-14 17:16:23,POINT (5.97513 52.20379)


In [17]:
gdf = GeoDataFrame(combined_dfs)

## Recalculate trajectory

In [18]:
traj = mpd.Trajectory(combined_dfs, 1, crs=4326)


In [19]:
traj.df

,geometry
DateTimeOfPosition,
2023-04-01 09:19:54,POINT (5.26999 53.38927)
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
2023-04-01 09:23:53,POINT (5.28571 53.39155)
...,...
2023-06-14 17:14:23,POINT (5.97857 52.19321)
2023-06-14 17:15:23,POINT (5.97986 52.20018)
2023-06-14 17:16:23,POINT (5.97513 52.20379)


In [20]:
traj_col = mpd.StopSplitter(traj).split(max_diameter=200, min_duration=timedelta(seconds=180))

In [21]:
data = {}
start_addrs = []
end_addrs = []
start_timestamps = []
end_timestamps = []
start_longs_and_lats = []
end_longs_and_lats = []
distances = []
durations = []

for idx, trajectory in enumerate(traj_col.trajectories):
        
    start_time = trajectory.df.index[0]
    end_time = trajectory.df.index[-1]
    
    start_timestamps.append(start_time)
    end_timestamps.append(end_time)
    
    #durations.append(str(end_time - start_time).split("days")[-1].split(".")[0])
    durations.append(end_time - start_time)

    start_xy = (trajectory.df["geometry"].x[0], trajectory.df["geometry"].y[0])
    end_xy = (trajectory.df["geometry"].x[-1], trajectory.df["geometry"].y[-1])
    
    
#     start_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[0]))
#     end_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[-1]))

    start_longs_and_lats.append(start_xy)
    end_longs_and_lats.append(end_xy)
    
    distances.append(haversine(start_xy, end_xy, unit=Unit.KILOMETERS))
    
# data["start_trip_addr"] = start_addrs
# data["end_trip_addr"] = end_addrs

data["start_trip_time"] = start_timestamps
data["end_trip_time"] = end_timestamps

data["start_gps_point"] = start_longs_and_lats
data["end_gps_point"] = end_longs_and_lats

data["trip_distance_in_km"] = distances

data["trip_duration"] = durations



In [22]:
final_df = pd.DataFrame(data)

In [23]:
final_df.sort_values("trip_distance_in_km")

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
242,2023-05-23 14:02:23,2023-05-23 14:02:54,"(5.138236, 51.805305)","(5.138526, 51.80536)",0.032817,0 days 00:00:31
279,2023-05-31 10:09:19,2023-05-31 10:11:19,"(4.266264, 51.948608)","(4.266095, 51.949544)",0.105478,0 days 00:02:00
176,2023-05-12 10:19:04,2023-05-12 10:19:36,"(5.475605, 52.230535)","(5.476698, 52.230639)",0.122080,0 days 00:00:32
283,2023-05-31 12:01:47,2023-05-31 12:02:46,"(4.302497, 52.107512)","(4.302994, 52.106496)",0.125481,0 days 00:00:59
205,2023-05-17 08:47:09,2023-05-17 08:48:09,"(6.36022, 51.903295)","(6.359983, 51.901352)",0.216333,0 days 00:01:00
...,...,...,...,...,...,...
13,2023-04-03 17:14:40,2023-04-03 18:47:38,"(5.217039, 53.357044)","(5.973632, 52.203424)",152.892231,0 days 01:32:58
339,2023-06-13 09:29:39,2023-06-13 11:13:35,"(6.002124, 51.185532)","(4.84507, 51.932242)",152.922515,0 days 01:43:56
171,2023-05-10 15:45:33,2023-05-10 17:39:31,"(4.610482, 51.561118)","(5.973216, 52.203351)",167.383830,0 days 01:53:58
285,2023-05-31 15:02:56,2023-05-31 16:46:34,"(4.435926, 52.21384)","(5.973557, 52.203344)",170.980953,0 days 01:43:38


In [24]:

cleaned_df = final_df[(final_df['trip_duration'].dt.seconds >= 180) & (final_df['trip_distance_in_km'] >= 0.2)]
#cleaned_df = final_df[final_df['trip_distance_in_km'] >= 0.2]


In [25]:
cleaned_df["trip_duration"].describe()

count                          290
mean     0 days 00:35:43.124137931
std      0 days 01:00:08.352466274
min                0 days 00:03:00
25%         0 days 00:12:39.750000
50%         0 days 00:27:19.500000
75%         0 days 00:46:19.250000
max                0 days 15:36:30
Name: trip_duration, dtype: object

In [26]:
cleaned_df.sort_values("trip_duration", ascending=False)

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
261,2023-05-26 19:57:02,2023-05-27 11:33:32,"(5.96819, 52.205382)","(5.919916, 52.206444)",5.369116,0 days 15:36:30
231,2023-05-22 11:27:05,2023-05-22 16:37:57,"(5.567698, 52.042289)","(5.379752, 52.158749)",24.554534,0 days 05:10:52
171,2023-05-10 15:45:33,2023-05-10 17:39:31,"(4.610482, 51.561118)","(5.973216, 52.203351)",167.383830,0 days 01:53:58
191,2023-05-16 07:25:00,2023-05-16 09:18:01,"(5.974655, 52.203694)","(5.06647, 51.586308)",121.931356,0 days 01:53:01
277,2023-05-31 06:25:58,2023-05-31 08:13:31,"(5.976286, 52.193825)","(4.236802, 51.857055)",196.985192,0 days 01:47:33
...,...,...,...,...,...,...
248,2023-05-24 09:47:18,2023-05-24 09:50:44,"(4.874577, 52.415514)","(4.883369, 52.410742)",1.111431,0 days 00:03:26
57,2023-04-15 11:08:43,2023-04-15 11:12:01,"(5.965575, 52.200122)","(5.974001, 52.203439)",1.006183,0 days 00:03:18
179,2023-05-13 08:21:44,2023-05-13 08:24:54,"(5.973286, 52.203734)","(5.967759, 52.200595)",0.705843,0 days 00:03:10
183,2023-05-13 11:17:06,2023-05-13 11:20:10,"(5.975727, 52.204582)","(5.980401, 52.200736)",0.671581,0 days 00:03:04


In [27]:
cleaned_df["ratio"] = cleaned_df.apply(lambda x:  (x["trip_distance_in_km"]*1000) / x["trip_duration"].seconds, axis=1)

In [28]:
cleaned_df.sort_values("ratio")

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration,ratio
261,2023-05-26 19:57:02,2023-05-27 11:33:32,"(5.96819, 52.205382)","(5.919916, 52.206444)",5.369116,0 days 15:36:30,0.095553
266,2023-05-28 14:22:36,2023-05-28 14:40:06,"(5.976167, 52.203965)","(5.973822, 52.203504)",0.265690,0 days 00:17:30,0.253038
12,2023-04-03 14:24:33,2023-04-03 14:48:33,"(5.215817, 53.361864)","(5.216866, 53.358065)",0.436552,0 days 00:24:00,0.303161
231,2023-05-22 11:27:05,2023-05-22 16:37:57,"(5.567698, 52.042289)","(5.379752, 52.158749)",24.554534,0 days 05:10:52,1.316456
78,2023-04-20 17:05:43,2023-04-20 17:12:07,"(6.102115, 52.502184)","(6.106574, 52.505095)",0.591123,0 days 00:06:24,1.539382
...,...,...,...,...,...,...,...
219,2023-05-19 06:29:11,2023-05-19 07:02:00,"(5.97396, 52.202439)","(5.376842, 52.154885)",66.604753,0 days 00:32:49,33.826690
291,2023-06-01 12:19:34,2023-06-01 13:07:45,"(6.167862, 52.545691)","(5.377182, 52.155059)",97.966721,0 days 00:48:11,33.886794
274,2023-05-30 15:57:32,2023-05-30 16:39:44,"(6.758112, 52.294812)","(5.984275, 52.24023)",86.258014,0 days 00:42:12,34.067146
309,2023-06-06 12:11:19,2023-06-06 12:47:19,"(6.311119, 53.167489)","(5.742669, 52.817516)",74.115107,0 days 00:36:00,34.312550


In [130]:
cleaned_df[cleaned_df["ratio"] > 1.0]

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration,ratio
0,2023-04-01 09:19:54,2023-04-01 09:23:53,"(5.269988, 53.389274)","(5.28571, 53.391549)",1.766263,0 days 00:03:59,7.390224
1,2023-04-01 11:07:38,2023-04-01 11:11:32,"(5.284403, 53.391597)","(5.269931, 53.388944)",1.635806,0 days 00:03:54,6.990626
2,2023-04-01 12:19:03,2023-04-01 12:26:22,"(5.270962, 53.389109)","(5.306682, 53.389323)",3.971959,0 days 00:07:19,9.047742
3,2023-04-01 12:37:27,2023-04-01 12:44:19,"(5.306676, 53.389332)","(5.269951, 53.388914)",4.083902,0 days 00:06:52,9.912382
4,2023-04-01 14:47:48,2023-04-01 14:52:01,"(5.272201, 53.389177)","(5.28636, 53.402797)",2.180136,0 days 00:04:13,8.617139
...,...,...,...,...,...,...,...
340,2023-06-13 12:26:03,2023-06-13 12:49:16,"(4.846854, 51.931568)","(4.917714, 51.74426)",22.197637,0 days 00:23:13,15.935131
341,2023-06-13 13:19:15,2023-06-13 13:49:15,"(4.916819, 51.742863)","(5.0676, 51.589485)",23.869822,0 days 00:30:00,13.261012
342,2023-06-13 14:48:58,2023-06-13 16:16:57,"(5.067325, 51.58887)","(5.973399, 52.203352)",121.556916,0 days 01:27:59,23.026504
343,2023-06-14 06:43:06,2023-06-14 07:38:21,"(5.974457, 52.203751)","(5.376879, 52.154919)",66.667055,0 days 00:55:15,20.110726


In [102]:
index_df.index[index_df["DateTimeOfPosition"] == "2023-05-26 19:57:02"].tolist()

[2922]

In [100]:
index_df = df.reset_index()

In [113]:
#index_df[2900:2925]

In [112]:
lat, long = (52.205382, 5.968190)
lat1, long1 = (52.206444, 5.919916)
haversine((lat, long), (lat1, long1))

3.291662790411985

## Finding Stop using IgnitionOn

In [366]:
df.sort_values("DateTimeOfPosition", inplace=True)

In [367]:
df

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
631414692,2023-04-01 08:50:30,0,53.389323,5.270153
631418242,2023-04-01 09:19:54,1,53.389274,5.269988
631418385,2023-04-01 09:20:53,1,53.389564,5.270365
631418504,2023-04-01 09:21:53,1,53.389952,5.274539
631418653,2023-04-01 09:22:53,1,53.391285,5.282144
...,...,...,...,...
647742547,2023-06-14 17:15:23,1,52.200175,5.979859
647742726,2023-06-14 17:16:23,1,52.203792,5.975125
647742890,2023-06-14 17:17:23,0,52.203289,5.972705


In [42]:

last_val = df.IgnitionOn.iloc[0]


In [125]:
dfcpy = df.copy()

In [163]:
dfcpy[:25]

,index,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,HaversineDistance
0,10696,631414692,2023-04-01 08:50:30,0,53.389323,5.270153,None
1,10695,631418242,2023-04-01 09:19:54,1,53.389274,5.269988,19.132581
2,10694,631418385,2023-04-01 09:20:53,1,53.389564,5.270365,52.805309
3,10693,631418504,2023-04-01 09:21:53,1,53.389952,5.274539,466.112333
4,10692,631418653,2023-04-01 09:22:53,1,53.391285,5.282144,858.422248
5,10691,631418767,2023-04-01 09:23:53,1,53.391549,5.285710,397.597612
6,10690,631418902,2023-04-01 09:24:53,1,53.392134,5.283950,206.143853
7,10689,631419027,2023-04-01 09:25:53,1,53.391772,5.284025,40.939971
8,10688,631419170,2023-04-01 09:26:53,0,53.392237,5.284695,90.560248
9,10687,631432987,2023-04-01 11:06:38,1,53.392189,5.284687,5.388609


In [368]:
df.reset_index(inplace=True)

In [372]:
df.drop_duplicates(subset="DateTimeOfPosition", inplace=True)

In [373]:
toggles = []
double_zeros = []
data = {}
for i in range(1, len(df)):
    if df.IgnitionOn.iloc[i] != df.IgnitionOn.iloc[i-1]:
        duration_diff = (df.DateTimeOfPosition.iloc[i] - df.DateTimeOfPosition.iloc[i-1])
        distance = haversine((df.lat.iloc[i], df.long.iloc[i]), (df.lat.iloc[i-1], df.long.iloc[i-1]), Unit.METERS)
        if  df.IgnitionOn.iloc[i] == 0:
            if (i+1) < len(df) and df.IgnitionOn.iloc[i] == df.IgnitionOn.iloc[i+1]:
                double_zeros.append(i+1)
            toggles.append((i, df.DateTimeOfPosition.iloc[i] - df.DateTimeOfPosition.iloc[i-1], df.DateTimeOfPosition.iloc[i]))
            data["DateTimeOfPosition"] = df.DateTimeOfPosition.iloc[i]
            data["IgnitionOn"] = df.IgnitionOn.iloc[i]
            #print("toggled")
    #print(df.IgnitionOn.iloc[i] != df.IgnitionOn.iloc[i-1], df.IgnitionOn.iloc[i])
    

In [255]:
#df.head(50) #2023-04-18 08:23:24

In [375]:
toggles

[(8, Timedelta('0 days 00:01:00'), Timestamp('2023-04-01 09:26:53')),
 (14, Timedelta('0 days 00:00:54'), Timestamp('2023-04-01 11:11:32')),
 (23, Timedelta('0 days 00:00:19'), Timestamp('2023-04-01 12:26:22')),
 (31, Timedelta('0 days 00:00:53'), Timestamp('2023-04-01 12:44:19')),
 (38, Timedelta('0 days 00:00:13'), Timestamp('2023-04-01 14:52:01')),
 (46, Timedelta('0 days 00:01:00'), Timestamp('2023-04-01 15:57:32')),
 (60, Timedelta('0 days 00:00:14'), Timestamp('2023-04-01 18:14:07')),
 (75, Timedelta('0 days 00:00:28'), Timestamp('2023-04-01 20:03:54')),
 (90, Timedelta('0 days 00:00:18'), Timestamp('2023-04-03 09:53:25')),
 (100, Timedelta('0 days 00:00:21'), Timestamp('2023-04-03 10:11:46')),
 (118, Timedelta('0 days 00:00:29'), Timestamp('2023-04-03 11:26:14')),
 (124, Timedelta('0 days 00:01:00'), Timestamp('2023-04-03 12:37:32')),
 (180, Timedelta('0 days 00:01:00'), Timestamp('2023-04-03 15:15:33')),
 (275, Timedelta('0 days 00:00:58'), Timestamp('2023-04-03 18:47:38')),
 (

In [374]:
len(toggles)

354

In [177]:
#sorted(toggles, key= lambda x: x[0])

In [41]:
df[8940:8950]

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
644958974,2023-06-03 01:16:02,0,52.203644,5.973617
645009214,2023-06-03 10:05:27,1,52.203595,5.973530
645009613,2023-06-03 10:06:26,1,52.203997,5.971517
645009860,2023-06-03 10:07:26,1,52.205637,5.968409
645009957,2023-06-03 10:07:53,0,52.205410,5.967754
645011387,2023-06-03 10:15:33,1,52.205271,5.967834
645011591,2023-06-03 10:16:32,1,52.204327,5.969279
645011763,2023-06-03 10:17:32,1,52.203842,5.973512
645011912,2023-06-03 10:18:16,0,52.203316,5.973202


In [376]:
double_zeros

[39,
 47,
 61,
 76,
 119,
 125,
 181,
 276,
 322,
 380,
 583,
 628,
 692,
 750,
 820,
 974,
 1000,
 1026,
 1121,
 1203,
 1267,
 1320,
 1363,
 1368,
 1388,
 1579,
 1618,
 1677,
 1896,
 1936,
 1956,
 2013,
 2059,
 2065,
 2111,
 2175,
 2228,
 2271,
 2318,
 2368,
 2595,
 2687,
 2747,
 2788,
 2829,
 2840,
 2893,
 2949,
 3042,
 3200,
 3331,
 3372,
 3491,
 3544,
 3583,
 3594,
 3699,
 3950,
 3994,
 4042,
 4144,
 4312,
 4385,
 4446,
 4486,
 4536,
 4574,
 4618,
 4645,
 4669,
 4695,
 4715,
 4765,
 4802,
 4914,
 5004,
 5391,
 5441,
 5508,
 5548,
 5654,
 5679,
 5686,
 5739,
 5793,
 5848,
 5964,
 6018,
 6117,
 6185,
 6265,
 6306,
 6497,
 6520,
 6541,
 6621,
 6654,
 6683,
 6703,
 6792,
 6856,
 7090,
 7205,
 7441,
 7509,
 7552,
 7608,
 7722,
 7736,
 7777,
 7798,
 7883,
 8023,
 8098,
 8265,
 8349,
 8486,
 8718,
 8758,
 8801,
 8870,
 8875,
 8885,
 8948,
 9014,
 9165,
 9381,
 9425,
 9468,
 9517,
 9571,
 9586,
 9710,
 9762,
 9803,
 9878,
 9908,
 9916,
 9966,
 10024,
 10521,
 10580,
 10625]

In [377]:

# for i in range(1, len(df)):
#     if df.IgnitionOn.iloc[i] != df.IgnitionOn.iloc[i-1]:
#         if  df.IgnitionOn.iloc[i] == 0:
#             if (i+2) < len(df) and df.IgnitionOn.iloc[i] == df.IgnitionOn.iloc[i+1] and df.IgnitionOn.iloc[i] == df.IgnitionOn.iloc[i+2]:
#                 print(i)
            

In [378]:
keep = [toggle[0] for toggle in toggles]

In [380]:
to_remove = list()
for val in df[df.IgnitionOn == 0].index.values:
    if val == 0: 
        continue
    if val not in keep:
        to_remove.append(val)

In [381]:
len(to_remove)

551

In [382]:
len(keep)

354

In [364]:
#df.head(50)

In [383]:
cleaned_df = df.drop(index=to_remove)

In [321]:
cleaned_df.sort_values("DateTimeOfPosition", ascending=True, inplace=True)

In [385]:
cleaned_df

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat
0,631414692,2023-04-01 08:50:30,0,53.389323,5.270153
1,631418242,2023-04-01 09:19:54,1,53.389274,5.269988
2,631418385,2023-04-01 09:20:53,1,53.389564,5.270365
3,631418504,2023-04-01 09:21:53,1,53.389952,5.274539
4,631418653,2023-04-01 09:22:53,1,53.391285,5.282144
...,...,...,...,...,...
10689,647742004,2023-06-14 17:12:23,1,52.182162,5.982590
10690,647742183,2023-06-14 17:13:23,1,52.188936,5.980637
10691,647742362,2023-06-14 17:14:23,1,52.193207,5.978572
10692,647742547,2023-06-14 17:15:23,1,52.200175,5.979859


## Get Stops

In [282]:
traj = mpd.Trajectory(cleaned_df[["DateTimeOfPosition", "long", "lat"]].sort_values("DateTimeOfPosition", ascending=False), "Allsetra", x='lat', y='long', t='DateTimeOfPosition', crs=4326)

detector = mpd.TrajectoryStopDetector(traj)
stops = detector.get_stop_points(min_duration=timedelta(seconds=180), max_diameter=200)


In [283]:
stops

,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
Allsetra_2023-04-01 08:50:30,POINT (5.27015 53.38932),2023-04-01 08:50:30,2023-04-01 09:20:53,Allsetra,1823.0
Allsetra_2023-04-01 09:23:53,POINT (5.28454 53.39195),2023-04-01 09:23:53,2023-04-01 11:07:38,Allsetra,6225.0
Allsetra_2023-04-01 11:11:32,POINT (5.27045 53.38903),2023-04-01 11:11:32,2023-04-01 12:19:03,Allsetra,4051.0
Allsetra_2023-04-01 12:26:22,POINT (5.30668 53.38933),2023-04-01 12:26:22,2023-04-01 12:37:27,Allsetra,665.0
Allsetra_2023-04-01 12:44:19,POINT (5.26995 53.38891),2023-04-01 12:44:19,2023-04-01 14:47:48,Allsetra,7409.0
...,...,...,...,...,...
Allsetra_2023-06-13 11:13:35,POINT (4.84507 51.93224),2023-06-13 11:13:35,2023-06-13 12:26:03,Allsetra,4348.0
Allsetra_2023-06-13 12:49:16,POINT (4.91754 51.74379),2023-06-13 12:49:16,2023-06-13 13:19:15,Allsetra,1799.0
Allsetra_2023-06-13 13:49:15,POINT (5.06728 51.58905),2023-06-13 13:49:15,2023-06-13 14:48:58,Allsetra,3583.0


In [323]:
cleaned_df.reset_index(inplace=True)
cleaned_df.drop("index", axis=1, inplace=True)
# Iterate through the DataFrame rows
for i in range(1, len(cleaned_df.index)):
    # Get the coordinates of the current and previous points
    lon1, lat1 = cleaned_df["long"].iloc[i], cleaned_df["lat"].iloc[i]
    lon2, lat2 = cleaned_df["long"].iloc[i-1], cleaned_df["lat"].iloc[i-1]
    
    # Calculate the Haversine distance between the points
    dist = haversine((lon1, lat1), (lon2, lat2), unit=Unit.KILOMETERS)
    duration = cleaned_df["DateTimeOfPosition"].iloc[i] - cleaned_df["DateTimeOfPosition"].iloc[i-1]
    # Update the Haversine distance in the DataFrame
    cleaned_df.at[i, 'haversine_distance'] = dist
    cleaned_df.at[i, 'duration'] = duration

In [326]:
cleaned_df.sort_values("haversine_distance", ascending=False)

,index,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,haversine_distance,duration
173,183,631817254,2023-04-03 17:15:39,1,53.177227,5.412910,23.863617,0 days 00:00:59
8284,8483,644338387,2023-05-31 15:42:56,1,52.324896,5.006019,4.263616,0 days 00:01:00
4828,4948,639463194,2023-05-09 13:32:37,1,52.324459,5.005575,4.192140,0 days 00:01:00
6435,6597,641542168,2023-05-19 06:38:12,1,52.187022,5.868476,4.179075,0 days 00:02:00
6440,6602,641542822,2023-05-19 06:44:12,1,52.191777,5.701592,3.985609,0 days 00:02:00
...,...,...,...,...,...,...,...,...
9118,9339,645683002,2023-06-06 13:05:19,1,52.817542,5.742647,0.000000,0 days 00:01:00
9119,9340,645683235,2023-06-06 13:06:19,1,52.817542,5.742647,0.000000,0 days 00:01:00
8143,8341,644238080,2023-05-31 11:23:30,1,52.010022,4.312554,0.000000,0 days 00:01:00
6938,7113,642465019,2023-05-23 12:27:05,1,51.775852,5.260684,0.000000,0 days 00:01:00


In [347]:
cleaned_df[168:180]

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,haversine_distance,duration
168,631783060,2023-04-03 15:12:33,1,53.357366,5.217147,0.000133,0 days 00:01:00
169,631783482,2023-04-03 15:13:33,1,53.357366,5.217147,0.000000,0 days 00:01:00
170,631783927,2023-04-03 15:14:33,1,53.356975,5.217310,0.044803,0 days 00:01:00
171,631784354,2023-04-03 15:15:33,0,53.356782,5.217236,0.022015,0 days 00:01:00
172,631817093,2023-04-03 17:14:40,1,53.357044,5.217039,0.031932,0 days 01:59:07
173,631817254,2023-04-03 17:15:39,1,53.177227,5.412910,23.863617,0 days 00:00:59
174,631817412,2023-04-03 17:16:39,1,53.177327,5.413247,0.025061,0 days 00:01:00
175,631817550,2023-04-03 17:17:39,1,53.176904,5.415634,0.165887,0 days 00:01:00
176,631817716,2023-04-03 17:18:39,1,53.174707,5.411680,0.359336,0 days 00:01:00
177,631817868,2023-04-03 17:19:39,1,53.172144,5.412588,0.291348,0 days 00:01:00


In [333]:
cleaned_df[8270:8295]

,index,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,haversine_distance,duration
8270,8469,644333607,2023-05-31 15:28:56,1,52.323447,4.766235,1.524785,0 days 00:01:00
8271,8470,644333949,2023-05-31 15:29:56,1,52.326539,4.790489,1.683788,0 days 00:01:00
8272,8471,644334258,2023-05-31 15:30:56,1,52.317909,4.809617,1.615838,0 days 00:01:00
8273,8472,644334608,2023-05-31 15:31:56,1,52.307599,4.826794,1.636385,0 days 00:01:00
8274,8473,644334946,2023-05-31 15:32:56,1,52.304549,4.841924,1.083145,0 days 00:01:00
8275,8474,644335305,2023-05-31 15:33:56,1,52.300249,4.858955,1.252852,0 days 00:01:00
8276,8475,644335633,2023-05-31 15:34:56,1,52.296955,4.876260,1.232442,0 days 00:01:00
8277,8476,644335981,2023-05-31 15:35:56,1,52.295334,4.887736,0.800967,0 days 00:01:00
8278,8477,644336323,2023-05-31 15:36:56,1,52.289964,4.907094,1.445623,0 days 00:01:00
8279,8478,644336715,2023-05-31 15:37:56,1,52.286257,4.931394,1.703439,0 days 00:01:00


## Using Ignition to detect stops and then trips using stops

In [343]:
zero_indexes = cleaned_df[cleaned_df["IgnitionOn"] == 0].index.to_list()

In [ ]:
idx = 0 
while idx < len(cleaned_df):
    if idx+1 > len(cleaned_df):
        break
    start_idx = zero_indexes[idx+1]
    end_idx = zero_indexes[]
    
    
        

In [341]:
cleaned_df

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,haversine_distance,duration
0,631414692,2023-04-01 08:50:30,0,53.389323,5.270153,NaN,NaT
1,631418242,2023-04-01 09:19:54,1,53.389274,5.269988,0.012223,0 days 00:29:24
2,631418385,2023-04-01 09:20:53,1,53.389564,5.270365,0.040803,0 days 00:00:59
3,631418504,2023-04-01 09:21:53,1,53.389952,5.274539,0.280134,0 days 00:01:00
4,631418653,2023-04-01 09:22:53,1,53.391285,5.282144,0.525633,0 days 00:01:00
...,...,...,...,...,...,...,...
10439,647742183,2023-06-14 17:13:23,1,52.188936,5.980637,0.764913,0 days 00:01:00
10440,647742362,2023-06-14 17:14:23,1,52.193207,5.978572,0.495336,0 days 00:01:00
10441,647742547,2023-06-14 17:15:23,1,52.200175,5.979859,0.779757,0 days 00:01:00
10442,647742726,2023-06-14 17:16:23,1,52.203792,5.975125,0.515598,0 days 00:01:00


In [344]:
zero_indexes

[0,
 8,
 14,
 23,
 31,
 38,
 45,
 58,
 72,
 83,
 93,
 111,
 116,
 171,
 265,
 310,
 367,
 418,
 462,
 513,
 571,
 615,
 680,
 687,
 717,
 737,
 806,
 815,
 825,
 829,
 893,
 924,
 954,
 958,
 981,
 1005,
 1010,
 1017,
 1055,
 1096,
 1100,
 1140,
 1176,
 1180,
 1194,
 1240,
 1292,
 1297,
 1334,
 1338,
 1357,
 1431,
 1492,
 1549,
 1587,
 1645,
 1649,
 1655,
 1674,
 1732,
 1827,
 1841,
 1864,
 1868,
 1902,
 1921,
 1976,
 2021,
 2026,
 2072,
 2081,
 2084,
 2087,
 2132,
 2184,
 2226,
 2271,
 2319,
 2404,
 2428,
 2436,
 2476,
 2479,
 2485,
 2519,
 2548,
 2581,
 2640,
 2700,
 2730,
 2732,
 2740,
 2745,
 2779,
 2783,
 2787,
 2837,
 2842,
 2891,
 2950,
 2984,
 3014,
 3058,
 3098,
 3142,
 3147,
 3167,
 3251,
 3271,
 3289,
 3311,
 3320,
 3374,
 3430,
 3433,
 3437,
 3454,
 3461,
 3481,
 3517,
 3527,
 3585,
 3610,
 3630,
 3667,
 3694,
 3696,
 3698,
 3707,
 3709,
 3711,
 3714,
 3726,
 3728,
 3735,
 3746,
 3753,
 3779,
 3781,
 3801,
 3811,
 3819,
 3876,
 3880,
 3903,
 3921,
 3937,
 3968,
 3973,
 4009

,GPSpoint,DateTimeOfPosition,IgnitionOn
UnitpositionID,,,
647751005,"N52.203256,E5.972714",2023-06-14 18:15:03,0
647742914,"N52.203289,E5.972705",2023-06-14 17:17:28,0
647742890,"N52.203289,E5.972705",2023-06-14 17:17:23,0
647742726,"N52.203792,E5.975125",2023-06-14 17:16:23,1
647742547,"N52.200175,E5.979859",2023-06-14 17:15:23,1
...,...,...,...
631418653,"N53.391285,E5.282144",2023-04-01 09:22:53,1
631418504,"N53.389952,E5.274539",2023-04-01 09:21:53,1
631418385,"N53.389564,E5.270365",2023-04-01 09:20:53,1


In [353]:
df.reset_index(inplace=True)

In [354]:
df[df["DateTimeOfPosition"] == "2023-04-03 17:15:39"]

,UnitpositionID,GPSpoint,DateTimeOfPosition,IgnitionOn
10513,631817254,"N53.177227,E5.412910",2023-04-03 17:15:39,1


In [355]:
df[10510:10518]

,UnitpositionID,GPSpoint,DateTimeOfPosition,IgnitionOn
10510,631817716,"N53.174707,E5.411680",2023-04-03 17:18:39,1
10511,631817550,"N53.176904,E5.415634",2023-04-03 17:17:39,1
10512,631817412,"N53.177327,E5.413247",2023-04-03 17:16:39,1
10513,631817254,"N53.177227,E5.412910",2023-04-03 17:15:39,1
10514,631817093,"N53.357044,E5.217039",2023-04-03 17:14:40,1
10515,631784359,"N53.356782,E5.217236",2023-04-03 15:15:34,0
10516,631784354,"N53.356782,E5.217236",2023-04-03 15:15:33,0
10517,631783927,"N53.356975,E5.217310",2023-04-03 15:14:33,1


In [216]:
traj

Trajectory Allsetra (2023-04-01 08:50:30 to 2023-06-14 17:17:23) | Size: 10381 | Length: 10047519.6m
Bounds: (3.752368, 51.132597, 6.922458, 53.40297)
LINESTRING (5.270153 53.389323, 5.269988 53.389274, 5.270365 53.389564, 5.274539 53.389952, 5.282144

In [217]:
splitted_traj = mpd.ObservationGapSplitter(traj).split(gap=timedelta(minutes=3))

In [218]:
dfs = [splitted.df for splitted in splitted_traj.trajectories]

combined_dfs = pd.concat(dfs)

In [219]:
combined_dfs

,geometry
DateTimeOfPosition,
2023-04-01 09:19:54,POINT (5.26999 53.38927)
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
2023-04-01 09:23:53,POINT (5.28571 53.39155)
...,...
2023-06-14 17:13:23,POINT (5.98064 52.18894)
2023-06-14 17:14:23,POINT (5.97857 52.19321)
2023-06-14 17:15:23,POINT (5.97986 52.20018)


In [220]:
data = {}
start_addrs = []
end_addrs = []
start_timestamps = []
end_timestamps = []
start_longs_and_lats = []
end_longs_and_lats = []
distances = []
durations = []

for idx, trajectory in enumerate(splitted_traj.trajectories):
        
    start_time = trajectory.df.index[0]
    end_time = trajectory.df.index[-1]
    
    start_timestamps.append(start_time)
    end_timestamps.append(end_time)
    
    #durations.append(str(end_time - start_time).split("days")[-1].split(".")[0])
    durations.append(end_time - start_time)

    start_xy = (trajectory.df["geometry"].x[0], trajectory.df["geometry"].y[0])
    end_xy = (trajectory.df["geometry"].x[-1], trajectory.df["geometry"].y[-1])
    
    
#     start_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[0]))
#     end_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[-1]))

    start_longs_and_lats.append(start_xy)
    end_longs_and_lats.append(end_xy)
    
    distances.append(haversine(start_xy, end_xy, unit=Unit.KILOMETERS))
    
# data["start_trip_addr"] = start_addrs
# data["end_trip_addr"] = end_addrs

data["start_trip_time"] = start_timestamps
data["end_trip_time"] = end_timestamps

data["start_gps_point"] = start_longs_and_lats
data["end_gps_point"] = end_longs_and_lats

data["trip_distance_in_km"] = distances

data["trip_duration"] = durations



In [221]:
splitted_df = pd.DataFrame(data)

In [223]:
splitted_df["trip_distance_in_km"].describe()

count    321.000000
mean      35.571719
std       37.991533
min        0.005036
25%        2.100296
50%       19.120770
75%       66.533134
max      196.985192
Name: trip_distance_in_km, dtype: float64

In [224]:
traj_col = mpd.StopSplitter(traj).split(max_diameter=200, min_duration=timedelta(seconds=180))

In [225]:
data = {}
start_addrs = []
end_addrs = []
start_timestamps = []
end_timestamps = []
start_longs_and_lats = []
end_longs_and_lats = []
distances = []
durations = []

for idx, trajectory in enumerate(traj_col.trajectories):
        
    start_time = trajectory.df.index[0]
    end_time = trajectory.df.index[-1]
    
    start_timestamps.append(start_time)
    end_timestamps.append(end_time)
    
    #durations.append(str(end_time - start_time).split("days")[-1].split(".")[0])
    durations.append(end_time - start_time)

    start_xy = (trajectory.df["geometry"].x[0], trajectory.df["geometry"].y[0])
    end_xy = (trajectory.df["geometry"].x[-1], trajectory.df["geometry"].y[-1])
    
    
#     start_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[0]))
#     end_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[-1]))

    start_longs_and_lats.append(start_xy)
    end_longs_and_lats.append(end_xy)
    
    distances.append(haversine(start_xy, end_xy, unit=Unit.KILOMETERS))
    
# data["start_trip_addr"] = start_addrs
# data["end_trip_addr"] = end_addrs

data["start_trip_time"] = start_timestamps
data["end_trip_time"] = end_timestamps

data["start_gps_point"] = start_longs_and_lats
data["end_gps_point"] = end_longs_and_lats

data["trip_distance_in_km"] = distances

data["trip_duration"] = durations



In [226]:
final_df = pd.DataFrame(data)

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
242,2023-05-23 14:02:23,2023-05-23 14:02:54,"(5.138236, 51.805305)","(5.138526, 51.80536)",0.032817,0 days 00:00:31
279,2023-05-31 10:09:19,2023-05-31 10:11:19,"(4.266264, 51.948608)","(4.266095, 51.949544)",0.105478,0 days 00:02:00
176,2023-05-12 10:19:04,2023-05-12 10:19:36,"(5.475605, 52.230535)","(5.476698, 52.230639)",0.122080,0 days 00:00:32
283,2023-05-31 12:01:47,2023-05-31 12:02:46,"(4.302497, 52.107512)","(4.302994, 52.106496)",0.125481,0 days 00:00:59
205,2023-05-17 08:47:09,2023-05-17 08:48:09,"(6.36022, 51.903295)","(6.359983, 51.901352)",0.216333,0 days 00:01:00
...,...,...,...,...,...,...
13,2023-04-03 17:14:40,2023-04-03 18:47:38,"(5.217039, 53.357044)","(5.973632, 52.203424)",152.892231,0 days 01:32:58
339,2023-06-13 09:29:39,2023-06-13 11:13:35,"(6.002124, 51.185532)","(4.84507, 51.932242)",152.922515,0 days 01:43:56
171,2023-05-10 15:45:33,2023-05-10 17:39:31,"(4.610482, 51.561118)","(5.973216, 52.203351)",167.383830,0 days 01:53:58
285,2023-05-31 15:02:56,2023-05-31 16:46:34,"(4.435926, 52.21384)","(5.973557, 52.203344)",170.980953,0 days 01:43:38


In [228]:
cleaned_df = final_df[(final_df['trip_duration'].dt.seconds >= 180) & (final_df['trip_distance_in_km'] >= 0.2)]


In [229]:
cleaned_df.sort_values("trip_duration", ascending=False)

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
261,2023-05-26 19:57:02,2023-05-27 11:33:26,"(5.96819, 52.205382)","(5.919922, 52.206456)",5.368478,0 days 15:36:24
231,2023-05-22 11:27:05,2023-05-22 16:37:57,"(5.567698, 52.042289)","(5.379752, 52.158749)",24.554534,0 days 05:10:52
171,2023-05-10 15:45:33,2023-05-10 17:39:31,"(4.610482, 51.561118)","(5.973216, 52.203351)",167.383830,0 days 01:53:58
191,2023-05-16 07:25:00,2023-05-16 09:18:00,"(5.974655, 52.203694)","(5.06647, 51.586308)",121.931356,0 days 01:53:00
277,2023-05-31 06:25:58,2023-05-31 08:13:31,"(5.976286, 52.193825)","(4.236802, 51.857055)",196.985192,0 days 01:47:33
...,...,...,...,...,...,...
57,2023-04-15 11:08:43,2023-04-15 11:12:01,"(5.965575, 52.200122)","(5.974001, 52.203439)",1.006183,0 days 00:03:18
179,2023-05-13 08:21:44,2023-05-13 08:24:54,"(5.973286, 52.203734)","(5.967759, 52.200595)",0.705843,0 days 00:03:10
273,2023-05-30 15:50:32,2023-05-30 15:53:32,"(6.751147, 52.301627)","(6.75687, 52.295296)",0.945354,0 days 00:03:00
183,2023-05-13 11:17:06,2023-05-13 11:20:06,"(5.975727, 52.204582)","(5.980404, 52.200734)",0.671979,0 days 00:03:00


In [236]:
cleaned_df[cleaned_df.DateTimeOfPosition == "2023-05-26 19:57:02"]

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat
7774,643436904,2023-05-26 19:57:02,1,52.205382,5.96819


In [246]:
cleaned_df[7540:7600]

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat
7724,643412190,2023-05-26 16:59:05,1,52.163934,5.496288
7725,643412359,2023-05-26 17:00:05,1,52.163471,5.502669
7726,643412545,2023-05-26 17:01:05,1,52.163302,5.504785
7727,643412713,2023-05-26 17:02:05,1,52.163221,5.505840
7728,643412885,2023-05-26 17:03:05,1,52.163174,5.506422
7729,643413059,2023-05-26 17:04:05,1,52.163102,5.507490
7730,643413228,2023-05-26 17:05:05,1,52.163097,5.507648
7731,643413392,2023-05-26 17:06:05,1,52.163005,5.508777
7732,643413548,2023-05-26 17:07:05,1,52.162864,5.510734
7733,643413723,2023-05-26 17:08:05,1,52.162667,5.516639


,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
0,2023-04-01 09:20:53,2023-04-01 09:23:53,"(5.270365, 53.389564)","(5.28571, 53.391549)",1.720386,0 days 00:03:00
1,2023-04-01 11:07:38,2023-04-01 11:11:32,"(5.284403, 53.391597)","(5.269931, 53.388944)",1.635806,0 days 00:03:54
2,2023-04-01 12:19:03,2023-04-01 12:26:22,"(5.270962, 53.389109)","(5.306682, 53.389323)",3.971959,0 days 00:07:19
3,2023-04-01 12:37:27,2023-04-01 12:44:19,"(5.306676, 53.389332)","(5.269951, 53.388914)",4.083902,0 days 00:06:52
4,2023-04-01 14:47:48,2023-04-01 14:52:01,"(5.272201, 53.389177)","(5.28636, 53.402797)",2.180136,0 days 00:04:13
...,...,...,...,...,...,...
340,2023-06-13 12:26:03,2023-06-13 12:49:16,"(4.846854, 51.931568)","(4.917714, 51.74426)",22.197637,0 days 00:23:13
341,2023-06-13 13:19:15,2023-06-13 13:49:15,"(4.916819, 51.742863)","(5.0676, 51.589485)",23.869822,0 days 00:30:00
342,2023-06-13 14:48:58,2023-06-13 16:16:57,"(5.067325, 51.58887)","(5.973399, 52.203352)",121.556916,0 days 01:27:59
343,2023-06-14 06:43:06,2023-06-14 07:38:21,"(5.974457, 52.203751)","(5.376879, 52.154919)",66.667055,0 days 00:55:15
